<a href="https://colab.research.google.com/github/Vaibhavs10/scratchpad/blob/main/langchain_hf_api_arxiv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install huggingface_hub chromadb langchain unstructured unstructured[local-inference]

In [2]:
from langchain.document_loaders import OnlinePDFLoader

loader = OnlinePDFLoader("https://arxiv.org/pdf/2304.03757.pdf")

In [3]:
documents = loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [4]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=350, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [7]:
from langchain.llms import HuggingFaceHub

flan_ul2 = HuggingFaceHub(repo_id="google/flan-ul2", model_kwargs={"temperature":0.1, "max_new_tokens":300})

In [8]:
from langchain.embeddings import HuggingFaceHubEmbeddings

embeddings = HuggingFaceHubEmbeddings()

You're using a different task than the one specified in the repository. Be sure to know what you're doing :)


In [9]:
from langchain.vectorstores import Chroma
db = Chroma.from_documents(texts, embeddings)

In [10]:
retriever = db.as_retriever()

In [14]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=flan_ul2, chain_type="stuff", retriever=retriever, return_source_documents=True)

In [16]:
query = "What is the title of this paper?"
result = qa({"query": query})

In [17]:
result

{'query': 'What is the title of this paper?',
 'result': 'Global Stability of Learning',
 'source_documents': [Document(page_content='number is deﬁned for every concept class H, even if H is not globally stable, whence\n\nρ(H) = 0. It is an asymptotic measure of how “stable” or “replicable” learning over\n\nH can potentially be. The larger the global stability parameter of H is, the more\n\n“replicable” learning over H can be.', metadata={'source': '/tmp/tmpz0wmwt06'}),
  Document(page_content='Deﬁnition 6 (Instability witness). An instability witness of size k for H ⊆ {±}X is\n\na function W : [k] × {±} → X × {±}. The witness W is composed of two functions\n\nWX : [k] × {±} → X and W± : [k] × {±} → {±}. It satisﬁes two requirements:\n\n(1) For every σ ∈ {±1}k, there is h ∈ H so that h(WX (j, σj)) = W±(j, σj) for\n\nall j ∈ [k].', metadata={'source': '/tmp/tmpz0wmwt06'}),
  Document(page_content='Let P denote the preﬁx of S consisting of the ﬁrst n1 examples.\n\nLet Q denote the suﬃx o